# Agent | Tools

**1. 도구 (Tool)**

https://docs.langchain.com/oss/python/integrations/tools/index#tools-and-toolkits

> "LLM이 사용할 수 있는 구체적인 기술이나 장비"

LLM은 기본적으로 학습된 데이터 내에서만 답변할 수 있으며, 실시간 정보나 정확한 수학 계산에는 취약하다. **Tool**은 이러한 한계를 보완하기 위해 LLM에게 쥐여주는 외부 기능이다.

* **역활:** 외부 API 호출, 웹 검색, 코드 실행, 파일 시스템 접근 등 LLM이 직접 할 수 없는 작업을 대신 수행한다.
* **예시:**
* `Google Search`: 최신 정보를 검색한다.
* `Calculator`: 정확한 수학 계산을 수행한다.
* `Python REPL`: 파이썬 코드를 작성하고 실행한다.



**2. 에이전트 (Agent)**

> "도구를 언제, 어떻게 사용할지 결정하는 두뇌"

**Agent**는 LLM을 추론 엔진(Reasoning Engine)으로 사용하여 사용자의 요청을 해결하기 위한 계획을 세우고 실행하는 주체이다. 단순히 정해진 코드를 순서대로 실행하는 것이 아니라, 상황에 따라 유연하게 행동을 결정한다.

* **역활:** 사용자의 질문을 분석하고, 어떤 **Tool**이 필요한지 판단(Thought)하고, 해당 도구를 실행(Action)한 뒤, 그 결과(Observation)를 보고 다음 행동을 결정하거나 최종 답변을 내놓는다.
* **작동 방식 (ReAct 패턴 예시):**
1. **질문:** "현재 서울 날씨에 맞는 옷차림 추천해줘."
2. **생각(Thought):** "서울의 현재 날씨를 먼저 알아야 한다." -> `Search` 도구 선택
3. **행동(Action):** `Search("서울 현재 날씨")` 실행
4. **관찰(Observation):** "서울 기온 5도, 맑음"이라는 결과 획득
5. **생각(Thought):** "5도면 코트나 패딩이 필요하다."
6. **최종 답변:** "현재 서울은 5도이므로 코트나 가벼운 패딩을 추천합니다."

**Agent와 Tools의 상호작용**

1. **Agent가 입력을 받음**: 사용자의 요청을 LLM으로 분석.
2. **적합한 Tool 선택**: 요청을 처리하는 데 가장 적합한 Tool을 선택.
3. **Tool 실행 및 결과 반환**: Tool을 실행하고 결과를 받아 사용자에게 응답.

In [2]:
%pip install langchain_openai langchain-community langchain-tavily langgraph wikipedia numexpr arxiv ddgs -Uqqq

Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv  # .env 파일의 환경변수 로드
import os                       # 환경변수 접근용

load_dotenv()                   # 현재 위치의 .env를 읽어와 환경변수로 등록
os.environ["OPENAI_API_KEY"] = os.getenv("openai_key")  # .env의 openai_key 값을 OPENAI_API_KEY로 등록
os.environ["LANGSMITH_TRACING"] = 'true'                # LangSmith 트레이싱 활성화
os.environ["LANGSMITH_ENDPOINT"] = 'https://api.smith.langchain.com'  # LangSmith API 엔드포인트 설정
os.environ["LANGSMITH_PROJECT"] = 'skn23-langchain'                   # LangSmith 프로젝트명 설정
os.environ["LANGSMITH_API_KEY"] = os.getenv("langsmith_key")          # .env의 langsmith_key 값을 LANGSMITH_API_KEY로 등록
os.environ["TAVILY_API_KEY"] = os.getenv("tavily_key")

## Tools

In [4]:
import importlib, pkgutil  # 모듈 동적 로드 / 패키지 탐색 유틸

package = importlib.import_module('langchain_community.tools')  # 패키지 로드

for module in pkgutil.iter_modules(package.__path__):  # 패키지 경로 아래의 하위 모듈들을 순회
    print(module.name)  # 각 모듈의 이름 출력

ainetwork
amadeus
arxiv
asknews
audio
azure_ai_services
azure_cognitive_services
bearly
bing_search
brave_search
cassandra_database
clickup
cogniswitch
connery
convert_to_openai
databricks
dataforseo_api_search
dataherald
ddg_search
e2b_data_analysis
edenai
eleven_labs
few_shot
file_management
financial_datasets
github
gitlab
gmail
golden_query
google_books
google_cloud
google_finance
google_jobs
google_lens
google_places
google_scholar
google_search
google_serper
google_trends
graphql
human
ifttt
interaction
jina_search
jira
json
memorize
merriam_webster
metaphor_search
mojeek_search
multion
nasa
nuclia
office365
openai_dalle_image_generation
openapi
openweathermap
passio_nutrition_ai
playwright
plugin
polygon
powerbi
pubmed
render
requests
riza
scenexplain
searchapi
searx_search
semanticscholar
shell
slack
sleep
spark_sql
sql_database
stackexchange
steam
steamship_image_generation
tavily_search
vectorstore
wikidata
wikipedia
wolfram_alpha
yahoo_finance_news
you
youtube
zapier
zenguar

In [5]:
from langchain_community.tools import WikipediaQueryRun         # 위키피디아 질의 실행 Tool
from langchain_community.utilities import WikipediaAPIWrapper   # 위키피디아 API 래퍼(검색/요약)

wiki_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())  # 위키 API 래퍼를 Tool에 연결
wiki_tool.run("LLM")  # 위키 검색/요약 결과 출력

c:\Users\Playdata\nlp\nlp_venv\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\Playdata\nlp\nlp_venv\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


"Page: Large language model\nSummary: A large language model (LLM) is a language model trained with self-supervised machine learning on a vast amount of text, designed for natural language processing tasks, especially language generation. The largest and most capable LLMs are generative pre-trained transformers (GPTs) that provide the core capabilities of modern chatbots. LLMs can be fine-tuned for specific tasks or guided by prompt engineering. These models acquire predictive power regarding syntax, semantics, and ontologies inherent in human language corpora, but they also inherit inaccuracies and biases present in the data they are trained on.\nThey consist of billions to trillions of parameters and operate as general-purpose sequence models, generating, summarizing, translating, and reasoning over text. LLMs represent a significant new technology in their ability to generalize across tasks with minimal task-specific supervision, enabling capabilities like conversational agents, cod

In [6]:
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent     # Tool 사용 에이전트 생성
from pprint import pprint    # 보기 좋게 출력

messages = [
    ('human', '올데이 프로젝트 멤버 알려줘!')
]

llm = init_chat_model('openai:gpt-4.1-mini')
# print(llm.invoke(message))

# LLM + Tool로 에이전트 구성
agent = create_agent(
    model = llm,
    tools = [wiki_tool]  # 위키피디아 검색/요약 Tool
)

response = agent.invoke({'messages': messages})  # messages를 dict로 넣어 에이전트 실행

pprint(response)

c:\Users\Playdata\nlp\nlp_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'messages': [HumanMessage(content='올데이 프로젝트 멤버 알려줘!', additional_kwargs={}, response_metadata={}, id='b270b0af-a256-4d13-8aba-6adeecb94632'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 91, 'total_tokens': 107, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_82dbabdb2c', 'id': 'chatcmpl-D63gu6HzZL9ResBNn38tfUcG6WTGa', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c3047-a8f1-7231-ae65-fac0e3cce7fd-0', tool_calls=[{'name': 'wikipedia', 'args': {'query': '올데이 프로젝트'}, 'id': 'call_w6X7cI5Oh7VwyhFFtS0x3ohN', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 91, 'out

In [7]:
response['messages'][-1].content  # response 내 messages 리스트의 마지막(최종 응답) content 출력

'올데이 프로젝트(AllDay Project) 멤버는 Annie, Tarzzan, Bailey, Woochan, Youngseo 이렇게 다섯 명입니다.'

### load_tools

**load_tools 사용가능 목록**

라이브러리를 통해 제공되는 외부 tool들을 langchain-community에서 통합하여 사용할 수 있다.
웬만한 기능들은 langchain 생태계 내에서 쉽게 쓸 수 있다.

https://docs.langchain.com/oss/python/integrations/providers/overview

https://docs.langchain.com/oss/python/integrations/tools

| 도구 이름        | 기능 예시               |
|------------------|------------------------|
| llm-math         | LLM 기반 수학 계산     |
| wikipedia        | 위키백과 검색          |
| serpapi          | 구글 검색 API          |
| requests_get     | HTTP GET 요청          |
| requests_post    | HTTP POST 요청         |
| arxiv            | arXiv 논문 검색        |
| pubmed           | PubMed 논문 검색       |
| dalle            | DALL-E 이미지 생성     |
| bing_search      | Bing 검색              |
| duckduckgo_search| DuckDuckGo 검색        |

### arxiv

In [8]:
from langchain_community.agent_toolkits.load_tools import load_tools  # 커뮤니티 제공 tool 로더

llm = init_chat_model('openai:gpt-4.1-mini')
tools = load_tools(['arxiv', 'wikipedia'])  # arXiv / 위키 Tool 로드
agent = create_agent(
    llm,
    tools,
    system_prompt='''
당신은 현명한 챗봇입니다.
주어진 도구를 적절히 활용하여, 사용자의 질문에 거짓없는 답변을 작성해주세요.
'''
)

messages = [
    ('human', '1706.03762 이 논문의 내용을 간단히 설명해줄래? (한글 답변)')
]

response = agent.invoke({'messages': messages})
pprint(response)
print(response['messages'][-1].content)

{'messages': [HumanMessage(content='1706.03762 이 논문의 내용을 간단히 설명해줄래? (한글 답변)', additional_kwargs={}, response_metadata={}, id='a149ad28-d4de-40f5-a771-3b6dd06937d4'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 214, 'total_tokens': 232, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_82dbabdb2c', 'id': 'chatcmpl-D63h44B0bDZ5IKHSXS0EAzQccORS1', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c3047-cfeb-74f1-b22c-db3949923a9f-0', tool_calls=[{'name': 'arxiv', 'args': {'query': '1706.03762'}, 'id': 'call_AYWANivCZUwtcTPUbfWD9GlW', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'i

### llm_math

In [9]:
from langchain_community.agent_toolkits.load_tools import load_tools  # 커뮤니티 제공 tool 로더

llm = init_chat_model('openai:gpt-4.1-mini')
tools = load_tools(['llm-math', 'wikipedia'], llm=llm)  # 수학도구 / 위키 Tool 로드 (수학도구는 llm이 필수)
agent = create_agent(
    llm,
    tools,
    system_prompt='''
당신은 현명한 챗봇입니다.
주어진 도구를 적절히 활용하여, 사용자의 질문에 거짓없는 답변을 작성해주세요.
단, 숫자계산은 반드시 llm-math 도구를 사용해야 합니다.
'''
)

response = agent.invoke({'messages': '3.5의 3제곱은 몇이야? 그리고 그 결과에 5를 곱해줘.'})
pprint(response)
print(response['messages'][-1].content)

{'messages': [HumanMessage(content='3.5의 3제곱은 몇이야? 그리고 그 결과에 5를 곱해줘.', additional_kwargs={}, response_metadata={}, id='228b8caf-8c0c-4532-9c23-2024e5993487'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 194, 'total_tokens': 214, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_82dbabdb2c', 'id': 'chatcmpl-D63h9V5pkFG3ZEvQXkHJySu10X1rH', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c3047-e5f4-7961-9ed9-860e12be05c4-0', tool_calls=[{'name': 'Calculator', 'args': {'__arg1': '3.5^3'}, 'id': 'call_l8B17bVBVcSrhcuOE3ubHBQA', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_t

### duckduckgo
https://docs.langchain.com/oss/python/integrations/tools/ddg

DuckDuckGo는 개인정보 추적 없이 웹 검색을 제공하는 검색 엔진으로,
LangChain에서는 이를 외부 최신 정보 검색용 Tool로 활용한다.

실시간 웹 검색 가능  
→ LLM의 knowledge cutoff 이후 이슈(뉴스, 화제, 트렌드)에 대응 가능  
로그인/API 키 불필요  
→ 실습·교육 환경에서 바로 사용 가능  
프라이버시 중심  
→ 사용자 검색 이력 추적 없음  

LangChain에서 제공하는 DuckDuckGo Tool 차이
- DuckDuckGoSearchRun
    - 검색 결과를 하나의 텍스트 요약으로 반환
    - 빠른 질의응답용에 적합
- DuckDuckGoSearchResults
    - 검색 결과를 리스트(제목, 링크, 스니펫 등 구조화) 형태로 반환
    - 에이전트가 여러 결과를 비교·판단해야 할 때 유리

In [10]:
# DDG 검색 Tool 2종류
from langchain_community.tools import DuckDuckGoSearchRun, DuckDuckGoSearchResults

ddgs = DuckDuckGoSearchRun()  # 검색결과를 텍스트 요약 형태로 반환
print(ddgs.invoke("현재 한국에서 가장 핫한 인물은?"))

ddgs2 = DuckDuckGoSearchResults()  # 검색 결과를 구조화된 리스트로 반환
print(ddgs2.invoke("현재 한국에서 가장 핫한 인물은?"))

6일 전 · 올해의 인물을 선정하는 것은 타임지 뿐만 아니라 유명 언론사, 과학계, 문화계등 다양한 곳에서 선정한다. 그 중 가장 오래되고 유명한 것이 타임지가 12월호에 기재하는 ... 2026. 1. 7. · 출처 나무위키 등 APT_LAP 손종원 Son Jong-won Jong- 41세 184 1984년생 배경 분당내 중학교 미국 고교유학 올 세인츠 데이 올세인츠데이스쿨 스쿨 (미국텍사스) ... 2025. 2. 14. · 2021년 '주목해야 할 남자배우' 1위를 차지한 이병헌이 4년 만에 다시 1위에 올라 관록을 입증했다면, 임지연과 고민시는 새롭게 5위권 안에 이름을 올렸고 변우석은 무려 ... 2025. 4. 17. · 오늘 가장 핫한 뉴스를 알아보는 핫클릭입니다. ▷ 로제, 미 타임지 '영향력 있는 100인' 선정 팝스타 브루노 마스와 협업한 곡 '아파트'로 세계적인 사랑을 받은 가수 ... 2일 전 · 배우 하정우와 차정원이 공식 열애를 인정했다. 결혼설은 부인했으며, 차정원 나이는 1989년생으로 하정우와 11세 차이다. 두 사람의 열애와 결혼설에 대중의 관심이 집중
snippet: Oct 28, 2025 · 2025년 기준, 한국에서 가장 주목받는 배우·가수·예능인 30인의 고향, 학력, 데뷔 계기, 대표 에피소드까지 한눈에 정리했습니다.2025년 현재 가장 핫 한 한국 연예인 30명의 프로필을 한눈에!, title: 요즘 가장 핫한 한국 연예인 30명 프로필 총정리, link: https://easy-quick.tistory.com/145, snippet: 15 마동석 영화배우 마동석은1971년 한국에서 태어나 18살 (1989)에 가족을 따라 미국으로 이민을 갔으며 미국 시민권을 취득했다. 미국에서 퍼스널 트레이너로 활동하다 2002년 영화 [천군] 오디션에 붙은 뒤 본격적으로 한국에서 배우 활동을 시작한다., title: KBS연예뉴스 TOP 10, link: https://kstar.kbs.co.kr/ranking.html?div=p

In [ ]:
# DDG로 에이전트 만들기
llm = init_chat_model('openai:gpt-4.1-mini')
tools = [ddgs2]
agent = create_agent(llm, tools)  # LLM + Tool로 에이전트 생성

# 메시지는 [(role, content)] 형태
response = agent.invoke({'messages': [('human', '현재 한국에서 가장 핫한 스몰토크 주제가 뭐야?')]})

pprint(response)
print(response['messages'][-1].content)  # 최종 답변 텍스트

{'messages': [HumanMessage(content='현재 한국에서 가장 핫한 스몰토크 주제가 뭐야?', additional_kwargs={}, response_metadata={}, id='74ac849e-8772-413a-846f-093141b918fd'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 90, 'total_tokens': 120, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_75546bd1a7', 'id': 'chatcmpl-D63hGU5RrhlHjT2Vnznu0PJeQfoKy', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c3047-ff2d-7540-9d87-4d65998afa41-0', tool_calls=[{'name': 'duckduckgo_results_json', 'args': {'query': '한국 핫한 스몰토크 주제 2024'}, 'id': 'call_LyiJAPOQbyUaQnFMZmlwpO5T', 'type': 'tool_call'}], invalid_tool_calls=[], usage_

### tavily-search

https://docs.langchain.com/oss/python/integrations/tools/tavily_search

In [ ]:
from langchain_tavily import TavilySearch  # Tavily 검색 툴

tavily_tool = TavilySearch(
    max_results = 3,            # 검색 결과 개수
    topic = 'general',          # general / news / finance 등
    include_images = True,      # 이미지 url을 포함해서 가져올지 여부
    search_depth = 'advanced'   # basic / advanced (advanced가 더 깊게 찾음)
)

tavily_tool.invoke('현재 서울에서 가장 핫한 지역은 어디야?')

{'query': '현재 서울에서 가장 핫한 지역은 어디야?',
 'response_time': 2.84,
 'follow_up_questions': None,
 'answer': None,
 'images': ['https://i.ytimg.com/vi/1nbCm5QiXls/maxresdefault.jpg',
  'https://i.ytimg.com/vi/YYTiwEYnPE0/maxresdefault.jpg',
  'https://scs-phinf.pstatic.net/MjAyNDA4MDFfMTkw/MDAxNzIyNTEwNjIzODY5.3u_AEdip5IrIaQS1AxX5zsKkqJhfiMAlqfhwEnVibqMg.5DuL5TghWyoEmiC5FyozowSH_EtKMncwoTitOALll4kg.PNG/image.png?type=w800',
  'https://ak-d.tripcdn.com/images/1mi4o224x8tk752vd5E8F_W_640_0_R5_Q80.jpg?proc=source/trip',
  'https://img1.daumcdn.net/thumb/R800x0/?scode=mtistory2&fname=https://blog.kakaocdn.net/dn/cLLeeM/btsMZqKiyA5/Oda4lkNt0kNO3LIdlcxDz0/img.jpg'],
 'results': [{'url': 'https://magazine.cheil.com/54545',
   'title': '넥스트 핫플레이스, 2024년 주목할 동네는? - 제일 매거진',
   'content': '대한민국 핫플레이스 1세대부터 4세대까지\n\n서울의 핫플레이스는 크게 1~4세대로 볼 수 있다. 1세대는 교통거점으로 남녀노소 유동 인구가 많은 곳이다. 명동, 서울역, 영등포역 등이 대표적이다. 지하철 1, 2호선을 만들 때 1세대 핫플레이스를 고려하고 노선을 짰고 여전히 1세대 핫플레이스는 여전히 접근성에선 장점을 가진다.\n\n2세대 핫플레이스는 대학가와 지하철 노선이 연결된 젊

In [15]:
# Tavily로 에이전트 만들기
llm = init_chat_model('openai:gpt-4.1-mini')
tools = [tavily_tool]
agent = create_agent(llm, tools)  # LLM + Tool로 에이전트 생성

# 메시지는 [(role, content)] 형태
response = agent.invoke({'messages': [('human', '2026년 현재 요즘 한국 교육계에서 이슈는 뭐야?')]})

pprint(response)
print(response['messages'][-1].content)  # 최종 답변 텍스트

{'messages': [HumanMessage(content='2026년 현재 요즘 한국 교육계에서 이슈는 뭐야?', additional_kwargs={}, response_metadata={}, id='5ad64ede-213b-4900-aa57-be6fa2b98d68'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 1203, 'total_tokens': 1236, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_75546bd1a7', 'id': 'chatcmpl-D63rElC3RQYpbnhvdCpHp1rzklko9', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c3051-6d8f-7261-a082-9cfaa491a1b3-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': '한국 교육계 2026년 이슈', 'time_range': 'year', 'topic': 'general'}, 'id': 'call_8SVNKvmQSs63Pe9PwAW8HCjB', 'type': 'tool_call

In [17]:
# Tavily로 미국주식시장 분석 에이전트 만들기
llm = init_chat_model('openai:gpt-4.1-mini')
tools = [tavily_tool]
# LLM + Tool + System Prompt로 에이전트 생성
agent = create_agent(
    llm,
    tools,
    system_prompt = '''
당신은 미국주식시장 분석봇입니다.
사용자가 요청한 기업에 대한 2026년 보고서를 직관적으로 분석해주세요.

# 출력형식
다음 내용을 포함해 표형식 출력 (분석기관별 레코드로 작성)

1. 분석기관명
2. 목표주가범위 (최저 ~ 최대)
3. 전망근거 키워드
4. 신뢰도 지수(1 ~ 10)
''')

# 메시지는 [(role, content)] 형태
response = agent.invoke({'messages': [('human', '2026년 엔비디아 주가 전망 분석해줘')]})

pprint(response)
print(response['messages'][-1].content)  # 최종 답변 텍스트

{'messages': [HumanMessage(content='2026년 엔비디아 주가 전망 분석해줘', additional_kwargs={}, response_metadata={}, id='588f2f84-56e2-46f4-bc50-b53b513de371'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 1298, 'total_tokens': 1336, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_75546bd1a7', 'id': 'chatcmpl-D63vJE1CPkOWNPQIwLP71LCaTb6rB', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c3055-48bd-7e91-9009-24c9c48928a7-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'NVIDIA stock price forecast 2026 analysis', 'search_depth': 'advanced', 'topic': 'finance', 'time_range': 'year'}, 'id': 'cal

In [18]:
# 최종 답변을 마크다운으로 렌더링해서 출력
from IPython.display import display, Markdown

display(Markdown(print(response['messages'][-1].content)))

2026년 엔비디아 주가 전망에 대해 여러 분석기관들의 내용을 종합한 결과는 아래와 같습니다.

| 분석기관명         | 목표주가범위 (최저 ~ 최대) | 전망근거 키워드                                           | 신뢰도 지수(1~10) |
|-------------------|--------------------------|--------------------------------------------------------|-----------------|
| 247WallSt.com     | $620 ~ $720              | AI 시장 성장, 데이터센터 매출 급증, 자동차 산업 파트너십, 미국-중국 무역 제한 리스크 | 8               |
| Nasdaq.com        | $650 ~ $750              | 데이터센터 투자 증가 기대, 2026년 약 $2070억 매출 예상, 향후 5년 42% CAGR 성장 전망   | 9               |
| 유튜브 투자전문가 | $600 ~ $700              | 2025년 변동성 컸으나 장기 성장 기대, 강한 매출 성장세, AI 및 자율주행 분야 확장        | 7               |

요약:
- 엔비디아는 AI 칩 및 데이터센터 매출 성장과 자동차 자율주행 기술 협력 등 포트폴리오 다각화로 2026년에도 큰 폭의 성장이 예상됨.
- 다만, 미국과 중국 간 무역 제한 문제 등 지정학적 리스크는 주가 변동성을 키울 수 있음.
- 전반적으로 600~750달러 사이 목표주가가 제시되며, 성장 투자자에게 매력적이라는 평가가 우세함.


<IPython.core.display.Markdown object>

### Tool

In [ ]:
a = 10
print(eval("5 + 3 + a"))  # 문자열 '표현식'을 평가해 결과 반환
exec("b = 10")            # 문자열 '문장/코드' 실행
print(b)

18
10


In [ ]:
# @tool로 간단한 계산기 Tool 만들기
from langchain_core.tools import tool  # 함수를 Langchain Tool로 등록하는 데코레이터

@tool
def simple_calculator(query: str) -> str:
    """
    산술연산을 위한 간단한 계산기
    Args:
        query: 계산식
    Return:
        계산식 결과값

    Examples:
    - simple_calculator("5 + 3 - 2") -> "계산 결과: 6"
    - simple_calculator("4 ** 2 / 8") -> "계산 결과: 2"
    """
    try:
        result = eval(query)           # 문자열 계산식을 eval로 평가(결과 반환)
        return f"계산 결과: {result}"   # 결과를 문자열로 포맷해서 반환
    except Exception as e:
        return f"계산 오류: {str(e)}"

simple_calculator

StructuredTool(name='simple_calculator', description='산술연산을 위한 간단한 계산기\nArgs:\n    query: 계산식\nReturn:\n    계산식 결과값\n\nExamples:\n- simple_calculator("5 + 3 - 2") -> "계산 결과: 6"\n- simple_calculator("4 ** 2 / 8") -> "계산 결과: 2"', args_schema=<class 'langchain_core.utils.pydantic.simple_calculator'>, func=<function simple_calculator at 0x000001FE786A6D40>)

In [22]:
# simple_calculator로 계산 에이전트 만들기
llm = init_chat_model('openai:gpt-4.1-mini')
tools = [simple_calculator]
agent = create_agent(llm, tools)  # LLM + Tool로 에이전트 생성

# 메시지는 [(role, content)] 형태
response = agent.invoke({'messages': [('human', '3 * 8 + 7 계산해줘')]})

pprint(response)
print(response['messages'][-1].content)  # 최종 답변 텍스트

{'messages': [HumanMessage(content='3 * 8 + 7 계산해줘', additional_kwargs={}, response_metadata={}, id='a5b61b3c-b447-4a22-9d79-35fa6e7bda89'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 123, 'total_tokens': 144, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_75546bd1a7', 'id': 'chatcmpl-D646WIDy9VbqehL86hAGjwYxYDL9A', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c305f-e411-73a2-9d5c-48cf53907573-0', tool_calls=[{'name': 'simple_calculator', 'args': {'query': '3 * 8 + 7'}, 'id': 'call_lb6RGImfNXQ5gxznPYYSlAUe', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 

In [ ]:
# 계산이 필요없는 일반질문 (Tool없이 직접 답변)
response = agent.invoke({'messages': [('human', '김치볶음밥 레시피는?')]})

pprint(response)
print(response['messages'][-1].content)  # 최종 답변 텍스트

{'messages': [HumanMessage(content='김치볶음밥 레시피는?', additional_kwargs={}, response_metadata={}, id='85c97d15-a7a5-45a8-8885-38de03faf20b'),
              AIMessage(content='김치볶음밥 레시피를 알려드릴게요.\n\n재료:\n- 밥 2공기\n- 김치 1컵 (잘게 썬 것)\n- 돼지고기 또는 베이컨 100g (선택 사항)\n- 양파 1/2개 (다진 것)\n- 대파 1대 (다진 것)\n- 식용유 2큰술\n- 간장 1큰술\n- 고추장 1큰술 (취향에 따라 조절)\n- 참기름 1큰술\n- 깨소금 약간\n- 계란 1~2개 (선택 사항, 반숙 또는 프라이)\n\n만드는 방법:\n1. 팬에 식용유를 두르고 다진 양파와 대파를 볶아 향을 낸다.\n2. 돼지고기나 베이컨을 넣고 익을 때까지 볶는다.\n3. 잘게 썬 김치를 넣고 3~4분간 중불에서 볶아 김치의 맛이 우러나게 한다.\n4. 밥을 넣고 재료들이 잘 섞이도록 볶아준다.\n5. 간장과 고추장을 넣고 간을 맞추며 골고루 섞는다.\n6. 마지막으로 참기름을 둘러 향을 더해주고 깨소금을 뿌린다.\n7. 원한다면 계란을 프라이해서 올려 마무리한다.\n\n맛있게 드세요! 더 필요하시면 말씀해 주세요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 335, 'prompt_tokens': 125, 'total_tokens': 460, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens

In [24]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENWEATHER_API_KEY = os.getenv('openweather_key')

In [ ]:
# OpenWeather API를 활용한 현재 날씨 조회 Tool
import requests
import json

@tool
def get_current_weather(city_name='Seoul', units='metric'):
    """
    OpenWeather API를 사용해서 사용자가 지정한 도시의 현재 날씨 정보를 가져오는 함수

    Args:
        - city: str 날씨정보를 가져올 도시 이름. **반드시 영문으로 작성하세요.**
            - 변환예시:
                - 서울 -> Seoul
                - 충남, 충청남도 -> Chungcheongnam-do
                - 부산 -> Busan
        - units: str 온도단위를 설정하는 문자열
            - metric(기본값: 섭씨, 미터)
            - imperial(화씨, 야드)
    Return:
        - str: json 형식으로 변환된 현재 날씨 정보
    """
    
    url = f'https://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={OPENWEATHER_API_KEY}&units={units}'
    response = requests.get(url)  # OpenWeather API 호출
    data = response.json()        # JSON 응답을 Dict로 변환

    weather_info = {}

    if response.status_code == 200:    # 응답 코드 200이면 정상 처리
        weather_description = data['weather'][0]['description']  # 날씨 설명
        temp = data['main']['temp']                              # 온도
        temp_feels_like = data['main']['feels_like']             # 체감 온도
        humidity = data['main']['humidity']                      # 습도

        weather_info = {
            'city': city_name,
            'description': weather_description,
            'temperature': temp,
            'temperature_feels_like': temp_feels_like,
            'humidity': humidity
        }
    else:  # 정상 응답이 아니면 실패 처리
        weather_info = {
            'city': city_name,
            'description': 'Not Found',
            'temperature': 'Not Found',
            'temperature_feels_like': 'Not Found',
            'humidity': 'Not Found'
        }

    return json.dumps(weather_info)  # dict -> json 문자열로 변환해 반환

get_current_weather

StructuredTool(name='get_current_weather', description='OpenWeather API를 사용해서 사용자가 지정한 도시의 현재 날씨 정보를 가져오는 함수\n\nArgs:\n    - city: str 날씨정보를 가져올 도시 이름. **반드시 영문으로 작성하세요.**\n        - 변환예시:\n            - 서울 -> Seoul\n            - 충남, 충청남도 -> Chungcheongnam-do\n            - 부산 -> Busan\n    - units: str 온도단위를 설정하는 문자열\n        - metric(기본값: 섭씨, 미터)\n        - imperial(화씨, 야드)\nReturn:\n    - str: json 형식으로 변환된 현재 날씨 정보', args_schema=<class 'langchain_core.utils.pydantic.get_current_weather'>, func=<function get_current_weather at 0x000001FE78AAD8A0>)

In [26]:
# 날씨 Tool을 함께 사용하는 에이전트로 서울 날씨 기반 옷차림 추천하기
llm = init_chat_model('openai:gpt-4.1-mini')
tools = [simple_calculator, get_current_weather]
agent = create_agent(llm, tools)  # LLM + Tool로 에이전트 생성

# 메시지는 [(role, content)] 형태
response = agent.invoke({'messages': [('오늘 뭐 입을까? 서울 날씨 좀 보고 대답해 줘')]})

pprint(response)
print(response['messages'][-1].content)  # 최종 답변 텍스트

{'messages': [HumanMessage(content='오늘 뭐 입을까? 서울 날씨 좀 보고 대답해 줘', additional_kwargs={}, response_metadata={}, id='53ffac93-3045-4c32-bd60-f8e7f03b8cce'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 298, 'total_tokens': 319, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_75546bd1a7', 'id': 'chatcmpl-D64JQ2uQqTi01QImFHJawVfpwaqlg', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c306c-19fa-7501-89df-9d2e4a27cc0b-0', tool_calls=[{'name': 'get_current_weather', 'args': {'city_name': 'Seoul', 'units': 'metric'}, 'id': 'call_4NRSLrJSF2JiBSOy41EoRWVL', 'type': 'tool_call'}], invalid_tool_calls=[],

In [ ]:
# 한국 기준 현재 날짜/시각을 반환하는 Tool
from datetime import datetime  # 현재 시각 조회용
from pytz import timezone      # 타임존(KST) 지정용

@tool
def get_current_datetime(format: str='%Y-%m-%d %H:%M:%S') -> str:
    """
    한국(KST) 기준 현재 시각을 문자열로 반환하는 Tool
    Args:
        format: 날짜/시각 형식 지정
    Return:
        현재시각 문자열

    get_current_datetime() -> "2026-01-15 12:18:32"
    """
    kst = timezone('Asia/Seoul')                 # 한국 시간대 설정
    return datetime.now(kst).strftime(format)    # 현재 시각을 format에 맞춰 문자열로 반환

get_current_datetime  # Langchain Tool 객체

StructuredTool(name='get_current_datetime', description='한국(KST) 기준 현재 시각을 문자열로 반환하는 Tool\nArgs:\n    format: 날짜/시각 형식 지정\nReturn:\n    현재시각 문자열\n\nget_current_datetime() -> "2026-01-15 12:18:32"', args_schema=<class 'langchain_core.utils.pydantic.get_current_datetime'>, func=<function get_current_datetime at 0x000001FE78AF3A60>)

In [30]:
# 오늘 날짜/생년월일로 만나이를 계산하는 Tool 정의/호출
@tool
def calculate_age(today_date: str, birth_date: str) -> int:
    """
    오늘 날짜와 생년월일로 만나이를 계산하는 Tool
    Args:
        - today_date(str): 오늘 날짜 (yyyy-mm-dd형식)
        - birth_date(str): 생년월일 (yyyy-mm-dd형식)
    Return:
        - 계산된 만나이(int)
    """
    try:
        today = datetime.strptime(today_date, '%Y-%m-%d')
        birthday = datetime.strptime(birth_date, '%Y-%m-%d')

        age = today.year - birthday.year
        if (today.month, today.day) < (birthday.month, birthday.day):
            age -= 1
        return age
    except ValueError:
        return "날짜형식이 올바르지 않습니다. yyyy-mm-dd 형식으로 전달하세요."

calculate_age.invoke({'today_date': '2026-02-06', 'birth_date': '1900-01-01'})

126

In [ ]:
# 위키디피아 + 날짜/나이 tool을 붙인 에이전트로 유명인 나이 질문하기
llm = init_chat_model('openai:gpt-4.1-mini')
tools = load_tools(['wikipedia']) + [get_current_datetime, calculate_age]  # 외부 위키 Tool + 커스텀 Tool(현재시간, 나이계산) 결합

agent = create_agent(
    llm,
    tools,
    system_prompt= "당신은 똑똑한 챗봇입니다. 주어진 도구를 적절히 활용해 대답하세요."
)

response = agent.invoke({'messages': [('human', '현재 대한민국 대통령님의 나이는 몇살이야?')]},
                        config = {'recursion_limit': 10})  # 설정 : ReAct 멀티턴(툴 호출 반복) 최대 횟수 제한

pprint(response)
print(response['messages'][-1].content)  # 최종 답변 텍스트

{'messages': [HumanMessage(content='대한민국 대통령님의 나이는 몇살이야?', additional_kwargs={}, response_metadata={}, id='e9f48d87-022d-4ae6-b95a-7f4a71637f2f'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 284, 'total_tokens': 328, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_75546bd1a7', 'id': 'chatcmpl-D64sjmd495ydRuanXfyD2kNoc4B90', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c308d-7ef4-7183-a986-53aada3ce58c-0', tool_calls=[{'name': 'wikipedia', 'args': {'query': '대한민국 대통령'}, 'id': 'call_dEULJJOjnFniPm93xfzHrOjc', 'type': 'tool_call'}, {'name': 'get_current_datetime', 'args': {}, 'id': 'call_Bi

## Memory
agent의 checkpointer속성에 메모리객체를 대화내역을 저장한다.
- 임시저장 InMemorySaver()
- 영구저장 SqliteSaver()

### InMemorySaver

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver  # 메모리 기반 체크포인트(대화 상태 저장)

llm = init_chat_model('openai:gpt-4.1-mini')
tools = [TavilySearch()]

agent = create_agent(llm, tools, checkpointer=InMemorySaver())  # 체크포인트를 붙여 상대 저장이 가능하도록 agent 생성

response = agent.invoke(
    input = {'messages': [('human', '안녕 반가워. 난 홍길동이라고 해. 넌 누구니?')]},
    config = {'configurable': {'thread_id': '100'}}  # thread_id로 세션(대화 스레드) 식별
)

print(response['messages'][-1].content)

c:\Users\Playdata\nlp\nlp_venv\Lib\site-packages\pydantic\json_schema.py:2448: PydanticJsonSchemaWarning: Default value <class 'langchain_tavily.tavily_search.TavilySearchInput'> is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\Playdata\nlp\nlp_venv\Lib\site-packages\pydantic\json_schema.py:2448: PydanticJsonSchemaWarning: Default value <factory> is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


안녕하세요 홍길동님! 저는 여러분과 대화하고 도움을 주기 위해 만들어진 AI 챗봇입니다. 반가워요! 어떻게 도와드릴까요?


In [37]:
response = agent.invoke(
    input = {'messages': [('human', '아 그래 너는 챗봇이구나! 내 이름이 뭐라고?')]},
    config = {'configurable': {'thread_id': '100'}}
)
print(response['messages'][-1].content)

c:\Users\Playdata\nlp\nlp_venv\Lib\site-packages\pydantic\json_schema.py:2448: PydanticJsonSchemaWarning: Default value <class 'langchain_tavily.tavily_search.TavilySearchInput'> is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\Playdata\nlp\nlp_venv\Lib\site-packages\pydantic\json_schema.py:2448: PydanticJsonSchemaWarning: Default value <factory> is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


네, 맞아요! 당신의 이름은 홍길동이라고 하셨어요. 기억하고 있어요. 다른 질문이나 도움이 필요하시면 언제든지 말씀해 주세요!


In [38]:
response = agent.invoke(
    input = {'messages': [('human', '아 그래 너는 챗봇이구나! 내 이름이 뭐라고?')]},
    config = {'configurable': {'thread_id': '200'}}
)
print(response['messages'][-1].content)

c:\Users\Playdata\nlp\nlp_venv\Lib\site-packages\pydantic\json_schema.py:2448: PydanticJsonSchemaWarning: Default value <class 'langchain_tavily.tavily_search.TavilySearchInput'> is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
c:\Users\Playdata\nlp\nlp_venv\Lib\site-packages\pydantic\json_schema.py:2448: PydanticJsonSchemaWarning: Default value <factory> is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


안녕하세요! 아직 당신의 이름을 알지 못해요. 알려주시면 기억할게요! 이름이 뭐예요?


### SqliteSaver

In [ ]:
%pip install -Uq langgraph-checkpoint-sqlite  # Langgraph 상태 저장을 SQLite로 영속화하는 체크포인터 패키지

Note: you may need to restart the kernel to use updated packages.


In [44]:
from langgraph.checkpoint.sqlite import SqliteSaver  # SQLite 기반 체크포인터(Saver)
from pprint import pprint

llm = init_chat_model('openai:gpt-4.1-mini')
tools = [TavilySearch()]

# SQLite DB 연결을 컨텍스트로 관리
with SqliteSaver.from_conn_string('checkpoint.db') as checkpointer:
    checkpointer.setup()  # checkpoint/writes 테이블 생성 및 초기화

    agent = create_agent(llm, tools, checkpointer=checkpointer)  # 상태 저장소를 SQLite 사용

    response = agent.invoke(
        input = {'messages': [('human', 'Langchain에 대해 설명해 줘')]},
        config = {'configurable': {'thread_id': '100'}}  # thread_id=100 세션으로 저장/조회
    )

    pprint(response)
    print(response['messages'][-1].content)
    print('-------------------------------')

    response = agent.invoke(
        input = {'messages': [('human', 'Langgraph에 대해 설명해 줘')]},
        config = {'configurable': {'thread_id': '100'}}  # thread_id=100 세션으로 저장/조회
    )
    pprint(response)
    print(response['messages'][-1].content)

{'messages': [HumanMessage(content='Langchain에 대해 설명해 줘', additional_kwargs={}, response_metadata={}, id='c65ced26-dd38-48ad-b831-cb03e37452e9'),
              AIMessage(content='Langchain은 자연어 처리와 인공지능 응용 프로그램 개발을 돕기 위한 오픈소스 프레임워크입니다. 주로 언어 모델(예: GPT-3, GPT-4)을 효과적으로 활용하여 대화형 에이전트, 챗봇, 문서 검색, 자동 요약, 정보 추출 등 다양한 언어 기반 애플리케이션을 쉽게 만들 수 있도록 설계되었습니다.\n\nLangchain의 주요 특징은 다음과 같습니다:\n1. 모듈화된 구성 요소: 프롬프트 관리, 언어 모델 호출, 체인(Chain) 구성, 데이터와의 통합 등을 모듈화하여 개발자가 필요한 부분을 쉽게 조합 가능.\n2. 체인(Chain) 구축: 여러 작업을 순차적 또는 병렬로 연결해 복잡한 작업 흐름을 만들 수 있음.\n3. 외부 데이터 연결: 데이터베이스, 문서 저장소, API 등 외부 데이터와 연동하여 실시간 정보 활용 가능.\n4. 에이전트(Agent) 기능: 상황에 맞게 작업을 수행할 수 있도록 동작 계획과 결정 로직을 내장한 지능형 에이전트 개발 지원.\n5. 다양한 언어 모델과 호환: OpenAI, Hugging Face, Cohere 등 여러 언어 모델 제공자와 통합.\n\n즉, Langchain은 언어 모델을 단순히 호출하는 것을 넘어 복잡한 언어 처리 작업을 효과적으로 관리하고 확장할 수 있게 해 주는 도구입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 328, 'prompt_tokens': 1195, 'total_tokens': 1523, 'completion_tokens_details': {'ac

In [45]:
# SQLite 체크포인터로 재접속시에 이전 대화 상태를 복원해 이어서 응답하기
from langgraph.checkpoint.sqlite import SqliteSaver  # SQLite 기반 체크포인터(Saver)
from pprint import pprint

llm = init_chat_model('openai:gpt-4.1-mini')
tools = [TavilySearch()]

# 동일한 DB 파일로 다시 연결 (재접속)
with SqliteSaver.from_conn_string('checkpoint.db') as checkpointer:
    checkpointer.setup()  # checkpoint/writes 테이블 생성 및 초기화

    agent = create_agent(llm, tools, checkpointer=checkpointer)  # 상태 저장소를 SQLite 사용

    response = agent.invoke(
        input = {'messages': [('human', '알았다. 그럼 네가 말해준 내용들을 하나의 노트로 정리해줘.')]},
        config = {'configurable': {'thread_id': '100'}}  # thread_id=100 세션으로 저장/조회
    )

    pprint(response)
    print(response['messages'][-1].content)

{'messages': [HumanMessage(content='Langchain에 대해 설명해 줘', additional_kwargs={}, response_metadata={}, id='c65ced26-dd38-48ad-b831-cb03e37452e9'),
              AIMessage(content='Langchain은 자연어 처리와 인공지능 응용 프로그램 개발을 돕기 위한 오픈소스 프레임워크입니다. 주로 언어 모델(예: GPT-3, GPT-4)을 효과적으로 활용하여 대화형 에이전트, 챗봇, 문서 검색, 자동 요약, 정보 추출 등 다양한 언어 기반 애플리케이션을 쉽게 만들 수 있도록 설계되었습니다.\n\nLangchain의 주요 특징은 다음과 같습니다:\n1. 모듈화된 구성 요소: 프롬프트 관리, 언어 모델 호출, 체인(Chain) 구성, 데이터와의 통합 등을 모듈화하여 개발자가 필요한 부분을 쉽게 조합 가능.\n2. 체인(Chain) 구축: 여러 작업을 순차적 또는 병렬로 연결해 복잡한 작업 흐름을 만들 수 있음.\n3. 외부 데이터 연결: 데이터베이스, 문서 저장소, API 등 외부 데이터와 연동하여 실시간 정보 활용 가능.\n4. 에이전트(Agent) 기능: 상황에 맞게 작업을 수행할 수 있도록 동작 계획과 결정 로직을 내장한 지능형 에이전트 개발 지원.\n5. 다양한 언어 모델과 호환: OpenAI, Hugging Face, Cohere 등 여러 언어 모델 제공자와 통합.\n\n즉, Langchain은 언어 모델을 단순히 호출하는 것을 넘어 복잡한 언어 처리 작업을 효과적으로 관리하고 확장할 수 있게 해 주는 도구입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 328, 'prompt_tokens': 1195, 'total_tokens': 1523, 'completion_tokens_details': {'ac

In [ ]:
# SQLite 체크포인터(DB)에서 특정 thread_id의 대화 메시지 목록 조회/출력
with SqliteSaver.from_conn_string('checkpoint.db') as checkpointer:
    checkpointer_tuple = checkpointer.get_tuple({"configurable": {"thread_id": "100"}})  # thread_id=100의 최신 체크포인트 조회

    checkpointer_data = checkpointer_tuple.checkpoint            # 체크포인트 원본 데이터(dict)
    messages = checkpointer_data['channel_values']['messages']   # 저장된 메시지 리스트 추출

    for i, message in enumerate(messages, 1):
        msg_type = getattr(message, 'type', message.__class__.__name__)  # message.type이 있으면 사용, 없으면 클래스명 사용
        print(f'{i}: [{msg_type}] {message.content}')  # (번호/타입/내용)
        print()

1: [human] Langchain에 대해 설명해 줘

2: [ai] Langchain은 자연어 처리와 인공지능 응용 프로그램 개발을 돕기 위한 오픈소스 프레임워크입니다. 주로 언어 모델(예: GPT-3, GPT-4)을 효과적으로 활용하여 대화형 에이전트, 챗봇, 문서 검색, 자동 요약, 정보 추출 등 다양한 언어 기반 애플리케이션을 쉽게 만들 수 있도록 설계되었습니다.

Langchain의 주요 특징은 다음과 같습니다:
1. 모듈화된 구성 요소: 프롬프트 관리, 언어 모델 호출, 체인(Chain) 구성, 데이터와의 통합 등을 모듈화하여 개발자가 필요한 부분을 쉽게 조합 가능.
2. 체인(Chain) 구축: 여러 작업을 순차적 또는 병렬로 연결해 복잡한 작업 흐름을 만들 수 있음.
3. 외부 데이터 연결: 데이터베이스, 문서 저장소, API 등 외부 데이터와 연동하여 실시간 정보 활용 가능.
4. 에이전트(Agent) 기능: 상황에 맞게 작업을 수행할 수 있도록 동작 계획과 결정 로직을 내장한 지능형 에이전트 개발 지원.
5. 다양한 언어 모델과 호환: OpenAI, Hugging Face, Cohere 등 여러 언어 모델 제공자와 통합.

즉, Langchain은 언어 모델을 단순히 호출하는 것을 넘어 복잡한 언어 처리 작업을 효과적으로 관리하고 확장할 수 있게 해 주는 도구입니다.

3: [human] Langgraph에 대해 설명해 줘

4: [ai] LangGraph에 대해 알려진 공식 정보는 제한적입니다. 그러나 이름과 관련 문맥으로 유추해 보면, LangGraph는 Langchain과 유사하게 언어 모델을 활용하는 도구이거나, 혹은 언어 모델 기반의 작업을 그래프 형태로 시각화하거나 관리하는 프레임워크일 가능성이 있습니다.

Langchain이 언어 모델의 체인(chain) 구조를 활용해 작업을 연결한다면, LangGraph는 그런 작업이나 데이터 흐름을 그래프(graph) 형태로 구조화하여 보다 직관적으로 관리하거나 분석할 수 있게 도와주는 도구

1️⃣ 세션(메모리) 유지 방식
예: store = {}, ChatMessageHistory, InMemorySaver
- 특징
    - 서버 메모리에만 대화 상태를 저장
    - 서버 재시작/재배포 시 모두 사라짐
    - 구현이 가장 단순하고 빠름
- 사용 시기
    - 실습 / 데모 / PoC
    - 단일 서버, 짧은 대화
    - “지금 이 세션에서만 기억하면 되는” 경우
- 장단점
    - ✅ 속도 빠름, 구현 쉬움
    - ❌ 서버 내려가면 기억 소멸
    - ❌ 멀티 서버(스케일아웃) 불가능

2️⃣ SQLite 체크포인터
예: SqliteSaver, checkpoint.db
- 특징
    - 로컬 파일(DB)에 대화 상태 저장
    - 서버 재시작해도 대화 복원 가능
    - 설정/운영 부담이 거의 없음
- 사용 시기
    - 1대 서버 운영
    - “재접속 시 대화 이어가기”가 중요한 서비스
    - 내부 도구, 사내용 챗봇, 파일 기반 서비스
- 장단점
    - ✅ 재시작해도 대화 유지
    - ✅ 설정 간단 (파일 하나)
    - ❌ 동시접속/대량 트래픽에 취약
    - ❌ 운영·분석·확장성 한계

3️⃣ RDB (MySQL / PostgreSQL 등)
실무에서 가장 많이 쓰는 방식

- 특징
    - 대화 내역을 정규화된 테이블로 저장
    - 여러 서버가 공유 DB 사용 가능
    - 사용자/세션/대화/이력 분석까지 가능
- 사용 시기
    - 실서비스(운영 환경)
    - 로그인 사용자 기반 챗봇
    - 고객지원, 상담, 금융, 헬스케어, 교육 서비스
- 장단점
    - ✅ 서버 여러 대에서도 동일한 대화 유지
    - ✅ 로그/분석/감사/리포트 가능
    - ✅ 권한·보안·백업 체계화 가능
    - ❌ 설계/운영 비용 존재

- 요약하자면  
메모리 세션	: 빠르고 간단  
SQLite 같은 파일형 DB : 재접속 기억 + 운영 부담 최소  
RDB	같은 관계형 데이터베이스 : 확장성, 안정성, 분석, 운영  

- 서비스 구상 단계에서  
사용자별 히스토리 관리  
문제 발생 시 감사 로그  
대화 품질/모델 성능 분석  
요약/임베딩/재검색(RAG) 연계  
개인화 서비스(추천, 성향 파악)  
이걸 하려면 RDB 또는 그 이상(이벤트 로그, 데이터 웨어하우스) 가 필요

## Middleware
https://docs.langchain.com/oss/python/langchain/middleware

미들웨어를 통해 에이전트의 추론 과정 중간에 개입하여 내부 동작을 커스터마이징할 수 있다.
Agent를 세부적으로 커스터마이징하기 위한 대부분의 작업을 미들웨어로 할 수 있다.

- 대화 기록 요약
- 동작 중 사용자 입력 대기
- 특정 모델 또는 tool에 대한 호출 제약
- fallback
- PII(개인식별정보) 처리 등

### SummarizationMiddleware

In [ ]:
from langchain.agents.middleware import SummarizationMiddleware  # 대화내역 자동 요약 미들웨어

model = init_chat_model('openai:gpt-4.1-mini')          # 메인 응답 모델
summary_model = init_chat_model('openai:gpt-4.1-mini')  # 요약 전용 모델

middleware_summarize = SummarizationMiddleware(  # 요약 미들웨어 생성
    model = summary_model,         # 요약에 사용할 모델
    trigger = ('tokens', 1000),    # 누적 토큰이 1000개 이상일때 실행
    keep = ('messages', 1),        # 최근 1개 메시지는 제외
    summary_prompt= '다음 대화내역을 적절히 요약해주세요.\n\n{messages}'  # 요약 프롬프트 템플릿
)

agent = create_agent(
    model = model,
    tools = [],
    checkpointer = InMemorySaver(),    # 상태 저장소(메모리)
    middleware=[middleware_summarize]  # 미들웨어(요약 기능)
)

In [50]:
# 첫 번째 질문
response = agent.invoke(
    input = {'messages': [('human', '뮤지컬 Wikied의 내용을 Elphaba입장에서 서술해줘. Elphaba역으로 연극에 출연해야 해. 중요한 포인트를 짚어줘~')]},
    config = {'configurable': {'thread_id': '100'}}
)
pprint(response)
print(response['messages'][-1].content)
print('-----\n')

# 두 번째 질문
response = agent.invoke(
    input = {'messages': [('human', 'Elphaba가 Glinda를 어떤 심정으로 보는 게 좋을까?')]},
    config = {'configurable': {'thread_id': '100'}}
)
pprint(response)
print(response['messages'][-1].content)
print('-----\n')

# 세 번째 질문
response = agent.invoke(
    input = {'messages': [('human', '또 연기할 때는 신경써야 할 것이 있을까?')]},
    config = {'configurable': {'thread_id': '100'}}
)
pprint(response)
print(response['messages'][-1].content)
print('-----\n')

{'messages': [HumanMessage(content='뮤지컬 Wikied의 내용을 Elphaba입장에서 서술해줘. Elphaba역으로 연극에 출연해야 해. 중요한 포인트를 짚어줘~', additional_kwargs={}, response_metadata={}, id='c042c253-1964-4b78-b97c-2dc321ad0cd5'),
              AIMessage(content='물론입니다! 뮤지컬 **[Wicked]**의 엘파바(Elphaba) 입장에서 내용을 정리해 드릴게요. 연극에서 엘파바를 연기할 때 중요한 감정선과 핵심 포인트도 함께 짚어드리겠습니다.\n\n---\n\n### 엘파바(Elphaba)의 시점에서 본 **Wicked** 이야기\n\n나는 엘파바, 초록빛 피부를 가진 여자다. 어렸을 때부터 나는 다른 사람들과 달랐다. 겉모습 때문에 따돌림을 당했고, 늘 소외감을 느껴야 했다. 하지만 나는 내가 누구인지, 그리고 내가 무엇을 할 수 있는지 알았다. 그래서 나는 나만의 길을 찾으려 애썼다.\n\n---\n\n### 주요 내용과 감정 포인트\n\n1. **출발점 – 아주 특별한 아이**\n   - 나는 태어날 때부터 초록빛 피부 때문에 주목받았다. 남들은 나를 이상하게 봤고, 그 눈빛에서 나는 매번 상처를 받았다.\n   - 하지만 나는 내 안에 강한 힘이 있다는 것을 알았다. 내가 가진 마법은 두려움 대상이 아니라, 올바른 일에 쓸 수 있는 힘이라는 확신이 있었다.\n   - *연기 포인트:* 초반에는 상처받고 혼란스러운 감정, 동시에 내면 깊은 곳에 숨겨진 자신감과 의지를 보여줘야 한다.\n\n2. **쉐르푸르크 대학교에 입학하다**\n   - 학교에서 나는 글린다와 친구가 되었다. 우리는 전혀 다르지만, 서로를 이해하며 가까워졌다.\n   - 하지만 동시에 학교와 주변 사람들은 나를 ‘악마’라 부르고 두려워했다. 이중적인 시선에 혼란스러웠다.\n   - *연기 포인트:* 친구 글린다와의 우정을 통해 따뜻함과 희망

### PIIMiddleware
**PII (Personally Identifiable Information) 개인식별정보 처리**

https://docs.langchain.com/oss/python/langchain/middleware/built-in#pii-detection

In [62]:
from langchain.agents.middleware import PIIMiddleware  # PII(개인정보) 탐지/처리 미들웨어

middleware_email = PIIMiddleware(
    pii_type = 'email',     # 이메일 타입 PII
    detector = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}",  # 이메일 형식
    strategy = 'redact',    # 탐지 시 삭제 처리
    apply_to_input = True   # 사용자 입력에 대해 적용
)

middleware_credit_card = PIIMiddleware(
    pii_type = 'credit_card',  # 신용카드 번호 타입 PII
    detector = r'(?:\d{4}[-\s]?){3}\d{4}|\d{4}[-\s]?\d{6}[-\s]?\d{5}',  # 카드번호(16자리) 형식
    strategy = 'mask',         # 탐지 시 마스킹 처리
    apply_to_input = True
)

middleware_api_key = PIIMiddleware(
    pii_type = 'api_key',    # API 키 처리 PII
    detector = r"sk-[a-zA-Z0-9-_]{161}",  # OpenAI 키(예시)
    strategy = 'mask',
    apply_to_input = True
)

agent = create_agent(
    model = init_chat_model('openai:gpt-4.1-mini'),
    tools = [],
    middleware = [middleware_email, middleware_credit_card, middleware_api_key]  # 입력 PII 처리 미들웨어들 적용
)

In [52]:
response = agent.invoke({
    'messages': [('human', 'AWS 요금이 3억이 청구됐어. 내가 지금 개발자를 지원하는 학생임을 감안해서 금액을 깎아달라고 이메일을 써줘.')]
})

pprint(response)

{'messages': [HumanMessage(content='AWS 요금이 3억이 청구됐어. 내가 지금 개발자를 지원하는 학생임을 감안해서 금액을 깎아달라고 이메일을 써줘.', additional_kwargs={}, response_metadata={}, id='421adb9a-c6e4-4a96-9c3b-90e2c0841157'),
              AIMessage(content='아래는 AWS 요금 관련하여 금액 조정 요청을 드리는 이메일 예시입니다. 학생 신분임을 강조하며 최대한 공손하게 작성했습니다.\n\n---\n\n제목: AWS 요금 관련 지원 요청드립니다\n\n안녕하세요, AWS 고객지원팀 귀하\n\n저는 현재 개발자를 지원하는 학생 신분으로, 학습 및 프로젝트 목적으로 AWS 서비스를 사용하고 있습니다. 그런데 이번에 청구된 요금이 약 3억 원에 달하는 것을 확인하고 매우 당황스러운 상황입니다.\n\n학생으로서 제한된 예산 내에서 학습을 진행하는 점을 고려해주실 수 있으면 매우 감사하겠습니다. 혹시 이번 요금에 대해 감면이나 지원, 또는 결제 분할 등의 조치가 가능한지 문의드립니다.\n\n추가적으로 요금 내역에 대해 자세히 검토할 수 있도록 안내받고 싶습니다.\n\n항상 AWS 서비스를 잘 활용할 수 있도록 도와주셔서 감사합니다. 긍정적인 답변 기다리겠습니다.\n\n감사합니다.\n\n[이름]  \n[연락처]  \n[이메일 주소]\n\n---\n\n필요에 따라 본인의 정보와 상황을 조금 더 상세히 덧붙여 주세요. 도움이 되었길 바랍니다!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 258, 'prompt_tokens': 46, 'total_tokens': 304, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 're

In [ ]:
response = agent.invoke({
    'messages': [('human', '환불요청 메일 좀 써줘. 내 신용카드 정보는 1234-1234-1234-1234야. 최근 아마존 쇼핑몰 주문건 $200에 대해서 환불 요청해줘.')]
})

pprint(response)

{'messages': [HumanMessage(content='환불요청 메일 좀 써줘. 내 신용카드 정보는 ****-****-****-1234야. 최근 아마존 쇼핑몰 주문건 $200에 대해서 환불 요청해줘.', additional_kwargs={}, response_metadata={}, id='854e93af-d9a0-4968-a72f-d4be77bed450'),
              AIMessage(content='개인 신용카드 정보는 메일에 포함하지 않는 것이 안전합니다. 아래에 환불 요청 메일 예시를 작성해 드릴게요.\n\n---\n\n제목: 주문 건 환불 요청드립니다 (주문번호: [주문번호 입력])\n\n안녕하세요,  \n저는 최근 아마존 쇼핑몰에서 총 $200 상당의 주문을 하였습니다.  \n주문번호는 [주문번호 입력]이며, 해당 주문 건에 대해 환불을 요청드립니다.\n\n환불 절차 안내 및 처리 진행 부탁드리겠습니다.  \n혹시 추가로 필요한 정보가 있으면 알려주세요.\n\n감사합니다.\n\n[이름]  \n[연락처]  \n[이메일 주소]\n\n---\n\n주문번호 등 구체적인 정보를 확인하여 기입해 주세요. 신용카드 정보는 메일에 포함하지 않는 것이 좋습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 182, 'prompt_tokens': 54, 'total_tokens': 236, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name

## Streaming
*openai모델은 조직인증된 사용자에 한해서 stream기능을 사용할수 있다.*

In [ ]:
model = init_chat_model('openai:gpt-4.1-mini')
agent = create_agent(model)

# 스트리밍 모드 실행 (메시지 단위로 받음)
stream = agent.stream(
    input= {'messages': [('human', '역사상 가장 위대한 한국의 장군은 누구인가?')]},
    stream_mode = 'messages'
)

for chunk, metadata in stream:
    print(chunk.content, end='', flush=True)  # 조각을 즉시 이어붙여 출력

역사상 가장 위대한 한국의 장군으로는 일반적으로 **이순신 장군**이 널리 꼽힙니다. 이순신 장군은 조선 중기 임진왜란(1592~1598) 당시 뛰어난 전략과 지휘력으로 왜군을 여러 차례 격파하며 조선을 수호한 인물입니다. 특히, 한산도 대첩과 명량 해전 등에서 보여준 그의 해전 전략과 불굴의 용기는 한국 역사뿐만 아니라 세계 해전사에서도 높이 평가받고 있습니다.

물론 한국 역사에는 다양한 시대와 지역에서 활약한 훌륭한 장군들이 많지만, 이순신 장군은 그의 업적과 리더십, 국민적 상징성 면에서 가장 대표적인 인물로 손꼽힙니다.